In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
import os

In [2]:
cv=LeaveOneOut()

In [3]:
#X = np.loadtxt("1.txt")
#X=pd.read_csv("Kmer4.csv",header=None)
y=pd.read_csv("lable.txt",header=None)
y=np.array(y)
#print(X.shape)

In [4]:
def my_classifier1(X,y):
    print(datetime.datetime.now())
    param1=["best","random"]
    param_grid={"splitter":param1}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
    return grid_search.best_params_["splitter"]
def my_classifier2(p1,X,y):
    print(datetime.datetime.now())
    param2=np.array(range(3,10,1))
    param_grid={"max_depth":param2}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10,splitter=p1),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
    return grid_search.best_params_["max_depth"]
def my_classifier(p1,p2,X, y):
    print(datetime.datetime.now())
   # param2=["best","random"]   ##设置splitter范围
   # param3=np.array(range(3,10,1))   ##设置max_depth范围
    i=X.shape[1]
    param3=np.array(range(1,i,100))  ## 设置max_features范围
    param_grid={"max_features":param3}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
#     print(datetime.datetime.now())
#     print("best parameters:{}".format(grid_search.best_params_))                        #输出最优网格化参数
#     print("best cross-validation score:{:.3f}".format(grid_search.best_score_))         #成员提供优化过程中观察到的最好评分
#     print("best estimator:\n{}".format(grid_search.best_estimator_))
    Predict_Y=[]    #存放预测值
    Prob_Y=[]       #存放预测概率
    Really_Y=y  #存放真实标签
    for train_index,test_index in cv.split(X):
        estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2,max_features=grid_search.best_params_["max_features"])
        estimator.fit(X[train_index],y[train_index])
        Predict_Y.append(estimator.predict(X[test_index])[0])
        Prob_Y.append(estimator.predict_proba(X[test_index]))
    Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
    Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
    Prob_Y=pd.DataFrame(Prob_Y)
    Prob_Y.columns=["0","1"]
    confusion_matrix(Really_Y,Predict_Y)
    TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
    FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
    FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
    TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
    ACC=accuracy_score(Really_Y,Predict_Y)
    F1_score=f1_score(Really_Y,Predict_Y)
    AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
    Recall=recall_score(Really_Y,Predict_Y)
    Precision=precision_score(Really_Y,Predict_Y)
    S=p1
    D=p2
    M=grid_search.best_params_["max_features"]
#     print("ACC:",ACC)
#     print("f1_score:",F1_score)
#     print("AUC:",AUC)
    return TN,FN,FP,TP,ACC,Recall,Precision,F1_score,AUC,S,D,M

In [5]:
file_path = 'E:\datatest\Kmer\k2' # 文件夹目录
files = os.listdir(file_path)
results = []
transfer=StandardScaler()
for file in files:
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=np.loadtxt(path_whole)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1=my_classifier1(x,y)
    x2=my_classifier2(x1,x,y)
    TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M = my_classifier(x1,x2,x, y)
    result = np.array([file,TN,FN,FP,TP,Acc,Rec,Pre,F,AUROC,S,D,M])
    print(result)
    results.append(result)
print(results)
results = np.array(results)
#print(results.shape)
result_final = pd.DataFrame(results)
#print(result_final)
result_final.to_csv(file_path+'\\'+'result.csv')

##################################################
6.txt
**************************************************
2022-02-14 05:30:06.123330
2022-02-14 05:30:29.732164
2022-02-14 05:31:24.954965
['6.txt' '33' '8' '55' '77' '0.6358381502890174' '0.9058823529411765'
 '0.5833333333333334' '0.7096774193548387' '0.6225935828877005' 'random'
 '9' '501']
[array(['6.txt', '33', '8', '55', '77', '0.6358381502890174',
       '0.9058823529411765', '0.5833333333333334', '0.7096774193548387',
       '0.6225935828877005', 'random', '9', '501'], dtype='<U18')]
